![LU Logo](https://www.lu.lv/fileadmin/user_upload/LU.LV/www.lu.lv/Logo/Logo_jaunie/LU_logo_LV_horiz.png)


# Vidējā līmeņa Python

## Nodarbības saturs

Mēs apskatīsim sekojošas tēmas:

* Dekoratori (decorators)
* Konteksta pārvaldnieki (context managers)
* Ģeneratori un ģeneratoru īspieraksts (generators, generator expressions)

## Prasības priekšzināšanām

* Pamatzināšanas par Python - kas līdz šim ir šajā kursā apskatīts.

## Nodarbības mērķi

Nodarbības beigās Jums ir jāspēj:

* Izveidot un pielietot dekoratorus
* Izveidot un pielietot konteksta pārvaldniekus
* Izveidot un pielietot ģeneratorus un to īspierakstu

## 1. tēma - Dekoratori

**Python dekoratori** ļauj paplašināt vai mainīt izsaucamā objekta (piemēram, funkcijas vai metodes) darbību, neiejaucoties paša izsaucamā objekta saturā. Būtībā dekoratori iesaiņo vai "izdekorē" funkciju, ļaujot veikt pirms- un pēcapstrādes darbības pirms un pēc sākotnējā funkcijas izsaukuma.

Dekoratorus var pielietot, lai papildinātu esošās funkcijas un metodes ar tādu funkcionalitāti kā notikumu žurnāla pieraksts (logging), kešatmiņa vai piekļuves kontrole.

Python dekoratori tiek pielietoti, izmantojot "@" sintaksi pirms funkcijas vai metodes definīcijas:

```
@decorator_name
def my_function():
    ...
```


### 1.1. Augstākas kārtas funkcijas

Python funkcijas ir "pirmās klases" objekti ("first-class" objects). Tās var piešķirt mainīgajiem, nodot kā argumentus un atgriezt no funkcijām (tieši tāpat kā jebkuru citu objektu).

Augstākas kārtas funkcijas (higher-order functions) ir funkcijas, kas darbojas ar citām funkcijām: tās var pieņemt funkcijas kā argumentus, tās var atgriezt funkcijas vai arī darīt gan vienu, gan otru. Šajā piemērā `func_2(my_function)` ir augstākas kārtas funkcija.

In [1]:
def func_1(text):
    print("In func_1:", text)

def func_2(my_function):
    print("In func_2:")
    my_function("Hi!")

func_1("Hello there!")
print()

func_2(func_1)

In func_1: Hello there!

In func_2:
In func_1: Hi!


Te mēs redzam, ka `func_2()` izpilda jebkuru funkciju, kas tai tiek nodota kā arguments.

Funkcijas iekšienē var arī definēt jaunas funkcijas (sauktas par *iekšējām* vai *iekļautām* funkcijām), kā arī atgriezt funkciju:

In [2]:
def parent_fn():
    print("Inside the parent_fn() function.")

    def child_fn(text):
        print("Inside the child_fn() function:", text)

    # returning a function
    return child_fn

parent_fn() # this will show the returned function object

Inside the parent_fn() function.


<function __main__.parent_fn.<locals>.child_fn(text)>

In [3]:
# parent_fn() returns a function
my_fn = parent_fn()
print()

# we can execute the function returned by parent_fn()
my_fn("Hello!")

Inside the parent_fn() function.

Inside the child_fn() function: Hello!


### 1.2. Dekoratori

Funkciju dekoratori "iesaiņo" funkciju vai klases metodi un modificē vai paplašina tās funkcionalitāti.

Izmantojot *augstākas kārtas funkcijas*, mēs varam definēt funkciju, kas saņem citu funkciju kā argumentu un atgriež jaunu funkciju:

In [4]:
def my_decorator(func):
    
    # function that calls func() provided as an argument to my_decorator()
    def wrapper():
        
        print("Do something before the function is called.")

        # call the original function
        func()
        
        print("Do something after the function is called.")

    # returns the wrapper funcion
    return wrapper

In [5]:
# define a new function
def my_fn():
    print("Executing my_fn() function.")

my_fn()

Executing my_fn() function.


Decorators wrap a function, modifying its behavior:

In [6]:
# now we can re-define the function by "wrapping" it in decorator's inner function
my_fn = my_decorator(my_fn)

my_fn()

Do something before the function is called.
Executing my_fn() function.
Do something after the function is called.


In [7]:
# Python decorators let us decorate functions by using the "@" syntax

# @my_decorator means the same as my_fn2 = my_decorator(my_fn2)

@my_decorator
def my_fn2():
    print("In the original function.")

my_fn2()

Do something before the function is called.
In the original function.
Do something after the function is called.


Jūs varat arī definēt dekoratorus savos moduļos un pēc tam importēt un pielietot tos:

In [8]:
%%writefile decorator_module.py

def do_twice(func):

    def wrapper():
        func()
        func()

    # returns the wrapper funcion
    return wrapper

Writing decorator_module.py


In [9]:
from decorator_module import do_twice

@do_twice
def my_fn3():
    print("Hello, world!")

my_fn3()

Hello, world!
Hello, world!


#### Funkcijas ar argumentiem un atgriežamām vērtībām

Ja dekorētā funkcija pieņem argumentus vai atgriež vērtību, dekoratoram arī ir jāapstrādā šie argumenti un jānodod tālāk atgrieztā vērtība.

Dekorētā funkcija var saņemt patvaļīgu skaitu pozicionālo un atslēgvārdu argumentu. Lai iegūtu šo argumentu vērtības, mēs varam izmantot sintaksi `*args` (pozicionālajiem argumentiem) un `**kwargs` (atslēgvārdu argumentiem), kas ļauj apstrādāt mainīgu argumentu skaitu:

In [11]:
def do_twice_args(func):

    # supply arguments to the inner function
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)

    # returns the wrapper funcion
    return wrapper

In [12]:
@do_twice_args
def print_name(name):
    print(f"Name: {name}")

print_name("John")

Name: John
Name: John


---

Var dekorēt arī funkcijas, kas atgriež vērtību:

In [13]:
def log_runs(func):

    # supply arguments to the inner function
    def wrapper(*args, **kwargs):
        print("Before the function call.")
        result = func(*args, **kwargs)
        print("After the function call. Return value:", result)

        return result

    return wrapper

In [14]:
@log_runs
def multiply_10x(number, my_name="Nothing"):
    print(f"In multiply_10x function. Name: {my_name}")
    return number * 10

multiply_10x(150, my_name="Uldis")

Before the function call.
In multiply_10x function. Name: Uldis
After the function call. Return value: 1500


1500

---

Python funkcijas parasti zina savu nosaukumu un atribūtus, un Python var parādīt dokumentāciju (help), kas apraksta šo funkciju. Tomēr, kad funkcijas tiek dekorētas, tās "pazaudē" šo informāciju. Tā vietā tiek parādīta informācija par aptverošo (wrapper) funkciju:

In [15]:
multiply_10x

<function __main__.log_runs.<locals>.wrapper(*args, **kwargs)>

In [16]:
help(multiply_10x)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
    # supply arguments to the inner function



Šo uzvedību var labot izmantojot `functools.wraps` dekoratoru:

In [17]:
import functools

def log_runs2(func):

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Before the function call.")
        result = func(*args, **kwargs)
        print("After the function call. Return value:", result)

        return result

    return wrapper

In [18]:
@log_runs2
def multiply_15x(number):
    """Multiply the number by 15."""
    
    print("In multiply_15x function.")
    return number * 15

multiply_15x(150)

Before the function call.
In multiply_15x function.
After the function call. Return value: 2250


2250

Python tagad var pareizi parādīt funkcijas nosaukumu un tās dokumentāciju (help):

In [19]:
multiply_15x

<function __main__.multiply_15x(number)>

In [21]:
help(multiply_15x)

Help on function multiply_15x in module __main__:

multiply_15x(number)
    Multiply the number by 15.



### 1.3. Dekoratoru piemēri

Python dekoratorus var pielietot dažādiem mērķiem, tostarp:

- **Žurnalēšanai** (logging) - lai pierakstītu informāciju par funkcijas izpildi
- **Autorizācijai** – pārbaudot, vai lietotājam ir tiesības izmantot funkciju
- **Kešošanai** – "dārgu" funkciju izsaukumu rezultātu glabāšanai un saglabātā rezultāta atgriešanai pēc pieprasījuma
- **Validācijai** – funkcijas ievades vai izvades vērtību pārbaudei

Ja nepieciešams, vienai un tai pašai funkcijai var pielietot vairākus dekoratorus.

#### Flask tīmekļa ietvars

[Flask](https://flask.palletsprojects.com/en/2.3.x/) ir Python tīmekļa mikro-ietvars, kas izmanto dekoratorus maršrutu (route) definēšanai un citiem nolūkiem.

Šis ir vienkāršs Flask piemērs, kurš, piekļūstot vietnei, atgriež ziņu "Hello, Flask!". Tas izmanto `@app.route()` dekoratoru, lai saistītu funkciju ar atbilstošo tīmekļa URL (maršrutu), kuru tā apstrādā:

```
from flask import Flask

app = Flask(__name__)

@app.route("/")
def home():
    return "Hello, Flask!"
```

Mēs varam arī definēt jaunu dekoratoru, kas pārliecinās, ka lietotājs ir pieteicies sistēmā (logged in) pirms piekļuves konkrētam maršrutam:

```
from flask import Flask, g, request, redirect, url_for
import functools

app = Flask(__name__)

def login_required(func):
    """Make sure user is logged in before proceeding"""
    @functools.wraps(func)
    def wrapper_login_required(*args, **kwargs):
        if g.user is None:
            return redirect(url_for("login", next=request.url))
        return func(*args, **kwargs)
    return wrapper_login_required
```

Pēc tam mēs varam pielietot šo dekoratoru (tādējādi funkcijai būs divi dekoratori) lai pārliecinātos, ka lietotājs ir pieteicies sistēmā:

```
@app.route('/secret')
@login_required
def secret():
    return "Welcome to this secret webpage!"
```

#### Datu klases

Dekorators `@dataclass()`, kas tika ieviests Python 3.7, nodrošina ērtu veidu, kā deklarēt [**datu klases**](https://docs.python.org/3/library/dataclasses.html#dataclasses.dataclass). Datu klases galvenokārt satur datus, un tās apraksta savus atribūtus, izmantojot klases mainīgo tipa anotācijas.

```
from dataclasses import dataclass

@dataclass
class Point:
    x: float
    y: float
    z: float = 0.0

p = Point(1.5, 2.5)

# prints "Point(x=1.5, y=2.5, z=0.0)"
print(p) 
```

Šajā piemērā ir redzams kā šo dekoratoru pielieto datu klases definēšanai.

#### Kešošana (caching)

Dekoratorus var izmantot kešošanai, saglabājot funkciju izsaukumu rezultātus atmiņā.

Kad funkcija tiek izsaukta atkārtoti ar tiem pašiem argumentiem, dekorators nolasa rezultātu no atmiņas, nevis izpilda funkciju, tādējādi optimizējot tās veiktspēju un samazinot lieku aprēķinu daudzumu.

Šis paņēmiens, kuru sauc arī par **memoizāciju** (memoization), ir īpaši noderīgs "dārgām" vai rekursīvām funkcijām. Piemērs rekursīvai funkcijai, kas var gūt ieguvumus no kešošanas, ir rekursīvā Fibonači virknes funkcija (skat. piemēru).


Python standarta bibliotēkā ir iekļauta LRU (least-recently-used) kešošanas funkcionalitāte, kas ir pieejama kā [@functools.lru_cache]((https://docs.python.org/library/functools.html#functools.lru_cache)) dekorators.

In [22]:
import functools

@functools.lru_cache(maxsize=10)
def fibonacci(num):
    
    print(f"Calculating fibonacci({num})")
    
    if num < 2:
        return num
        
    return fibonacci(num - 1) + fibonacci(num - 2)

In [23]:
# "Calculating fibonacci" is printed every time the function is executed
fibonacci(10)

Calculating fibonacci(10)
Calculating fibonacci(9)
Calculating fibonacci(8)
Calculating fibonacci(7)
Calculating fibonacci(6)
Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)


55

In [24]:
# not printing "Calculating fibonacci" this time because the return value has already been cached
fibonacci(8)

21

## 2. tēma - Konteksta pārvaldnieki

**Konteksta pārvaldnieki** ļauj automātiski piešķirt un atbrīvot resursus, tad, kad tas ir nepieciešams, nodrošinot resursu piešķiršanas un atbrīvošanas darbību uzticamu izpildi.

Konteksta pārvaldniekiem ir dažādi pielietojumi, tostarp:
* **failu operācijas** - nodrošinot, ka atvērtie faili tiek pareizi aizvērti
* **datu bāzes savienojumi** — nodrošinot, ka datu bāzu savienojumi tiek pareizi aizvērti, apstiprināti (commit) vai atgriezti atpakaļ (rollback)
* **pavedienu drošība** — resursu piekļuves drošība (thread-safety), izmantojot bloķēšanas mehānismus

---

Konteksta pārvaldnieki parasti tiek lietoti kopā ar `with` komandu. Tā bieži tiek izmantota darbam ar failiem:

```
with open("test_file.txt", "r") as file:
    text = file.read()
```

Šeit komanda `with` nodrošina to, ka atvērtais fails tiek pareizi aizvērts, pat ja rodas kļūdas situācija (exception). Fails tiek aizvērts, kad programma iziet no `with` komandu bloka: `text = file.read()`

Failu var aizvērt arī manuāli, neizmantojot konteksta pārvaldnieku, kā parādīts nākamajā piemērā. Tomēr, ja `file.read()` izsaukuma laikā radīsies kļūdas situācija, tad fails netiks automātiski aizvērts.

```
file = open("test_file.txt", "r")
text = file.read()
file.close()
```

Tad, lai pareizi aizvērtu failu (arī kļūdas gadījumā), Jums būtu jāizmanto `try: ... finally:` komandas:

```
file = open("test_file.txt", "r")

try:
    content = file.read()

finally:
    file.close()
```

Konteksta pārvaldnieki vienkāršo resursu pārvaldību Python programmēšanas valodā, padarot kodu vieglāk lasāmu un mazāk pakļautu kļūdām.

---

**Konteksta pārvaldnieku** var īstenot divos veidos:

* izmantojot konteksta pārvaldnieka protokolu (interfeisu)
* izmantojot ģeneratora funkciju

**Konteksta pārvaldnieka protokols** ļauj definēt konteksta pārvaldnieku kā klasi, ieviešot tajā divas īpašas metodes: `__enter__` un `__exit__`. Šīs metodes ļauj izstrādātājiem uzstādīt (set up) un nojaukt (tear down) konteksta pārvaldnieka pārvaldītos resursus.

In [25]:
# we will duplicate the functionality of the built-in open() function
class FileOpener:
    
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None

    def __enter__(self):
        print(f"Opening: {self.filename}")
        self.file = open(self.filename, self.mode)
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            print(f"Closing: {self.filename}")
            self.file.close()

In [26]:
# now we use our custom FileOpener class as a context manager
# we can do so because FileOpener has __enter__() and __exit__() methods

with FileOpener("decorator_module.py", "r") as in_file:
    print("Reading the file.")
    content = in_file.read()
# here file is already closed just like with the built-in open() function

Opening: decorator_module.py
Reading the file.
Closing: decorator_module.py


---

Konteksta pārvaldniekus var izmantot, lai **pārvaldītu datu bāzes transakcijas**:

```
class DatabaseTransaction:
    def __init__(self, connection):
        self.connection = connection

    def __enter__(self):
        return self.connection.cursor()

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            self.connection.commit()
        else:
            self.connection.rollback()
```

Pieņemot, ka `conn` norāda uz jau uzstādītu datu bāzes savienojumu:

```
with DatabaseTransaction(conn) as cursor:
    cursor.execute('INSERT INTO table (col1, col2) VALUES (?, ?)', (val1, val2))
```

---

**Taimera konteksta pārvaldnieks** ļauj izmērīt laiku, kas nepieciešams koda bloka izpildei.

In [27]:
import time

class CodeTimer:
    def __enter__(self):
        self.start_time = time.time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end_time = time.time()
        elapsed_time = self.end_time - self.start_time
        print(f'Code took {elapsed_time:.2f} seconds to execute')


In [28]:
with CodeTimer():
    # some time-consuming operations
    result = [i**2 for i in range(1000000)]

Code took 0.03 seconds to execute


In [32]:
# Jupyter also has a "magic" keyword for measuring the duration of code execution.

%time result = [i**2 for i in range(1000000)]

print()

%timeit result = [i**2 for i in range(1000000)]

CPU times: user 25.4 ms, sys: 5.15 ms, total: 30.5 ms
Wall time: 29.7 ms

27.6 ms ± 318 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 3. tēma - Ģeneratori

Ģeneratoru funkcijas ļauj deklarēt funkcijas, kas darbojas kā iteratori (piemēram, tās var izmantot for ciklā).

Šīs funkcijas ģenerē (yield) vērtības katrai cikla iterācijai, nevis atgriež vienu vienīgu vērtību funkcijas beigās.

https://wiki.python.org/moin/Generators

* Ģeneratori (ģeneratoru funkcijas)
* Ģeneratoru īspieraksts (generator expressions)

Ģeneratora funkcijas tiek definētas tāpat kā citas Python funkcijas
- izņemot to, ka tās izmanto `yield` komandu, lai atgrieztu vērtības (piemēram, katrai cikla iterācijai), kamēr šī ģeneratora funkcija darbojas.

---

Ģeneratori:
- ir vienkāršāki nekā parastās funkcijas, kas veic to pašu uzdevumu
- izmanto mazāk atmiņas, jo atgrieztās vērtības tiek aprēķinātas uz vietas, bez nepieciešamības saglabāt visas vērtības sarakstā
- ir "slinki" – tie ģenerē vērtības tikai tad, kad tās tiek pieprasītas

Ģeneratori var ģenerēt datus, kas ir milzīgi vai pat bezgalīgi.

#### 3.1. Piemērs

In [33]:
# First n numbers - using a regular function

def first_n(n):
    '''Build and return a list'''
    num, nums = 0, []
    
    while num < n:
        nums.append(num)
        num += 1
        
    return nums

In [34]:
res = first_n(1000_000)

# how much memory does it use?
import sys
print(f"Memory used: {sys.getsizeof(res)} bytes")
print()

# sum of all the numbers
print(sum(res))

Memory used: 8448728 bytes

499999500000


In [36]:
# First n numbers - using a generator function

def first_n_gen(n):
    
    num = 0
    
    while num < n:
        yield num
        num += 1

# Generator yields items (as they are requested) instead of returning a list 
# at the end of the function

In [37]:
res_gen = first_n_gen(1_000_000)

# the result is a generator
print(type(res_gen))

print()
print(f"Size of res_gen in bytes: {sys.getsizeof(res_gen)}")

<class 'generator'>

Size of res_gen in bytes: 192


In [38]:
# generator generates values one-by-one, on demand

# calculate the sum of all the values
print(sum(res_gen))

499999500000


In [39]:
# once all generator values have been requested,
# it is "used up" - no values remain to return

# so we can not iterate through a generator twice in a row

print(sum(res_gen))
print()

# but we can create a new generator object and iterate through it

print(sum(first_n_gen(1_000_000)))

0

499999500000


---

#### 3.2 – Ģeneratoru izveide

Iteratori ir objekti, kas ļauj "pārvietoties" (iterēt) pa datu kolekciju elementiem.

Ģeneratoru izveidei tiek pielietots iteratoru protokols:
- `__iter__()` tiek izsaukta, lai inicializētu iteratoru. Tai ir jāatgriež iteratora objekts (parasti tā atgriež `self`)
- `__next__()` tiek izmantota, lai iterētu pa iteratoru (tā atgriež nākamo iteratora vērtību)
- kad datu plūsma ir beigusies, iteratoram ir jāatgriež `StopIteration` izņēmums (exception).

Kad ģenerators ir izveidots, jūs varat iterēt pa to, izmantojot `for` ciklu, vai iegūt vērtības tieši, izmantojot Python `next()` funkciju.

In [40]:
res = first_n_gen(10)

for i in res:
    print(i)

0
1
2
3
4
5
6
7
8
9


In [41]:
# we could implement an iterator directly
# but that would require more code than necessary

# generators make this code simpler and easier to understand

class first_n_iter:

    def __init__(self, n):
        self.n = n
        self.num = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.num < self.n:
            cur, self.num = self.num, self.num+1
            return cur
            
        raise StopIteration()

print(sum(first_n(1_000_000)))

499999500000


In [43]:
# methods and attributes that a generator object has:

my_data = first_n_gen(5)

for name in dir(my_data):
    print(name)

__class__
__del__
__delattr__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__iter__
__le__
__lt__
__name__
__ne__
__new__
__next__
__qualname__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
close
gi_code
gi_frame
gi_running
gi_suspended
gi_yieldfrom
send
throw


In [44]:
# Generators are lazy and you can only iterate through them once

next_val = next(my_data)
print(next_val)

0


In [45]:
print(next(my_data))
print(next(my_data))
print(next(my_data))
print(next(my_data))

1
2
3
4


In [47]:
# Data has finished, this will raise a StopIteration exception
#  - uncomment the next line to see it
#print(next(my_data))

In [48]:
# for loops know how to use the iterator protocol
# (e.g. they handle the StopIteration exception)

for item in first_n_gen(5):
    print(item)

0
1
2
3
4


In [49]:
# let's add some print() calls

def my_gen_fn(n):
    print("In generator function")

    num = 0
    
    while num < n:
        print("- before yield")
        yield num
        print("- after yield")
        
        num += 1

    print("End of the function")

In [50]:
# this will call the my_gen_fn() function
# what will it print?

my_gen = my_gen_fn(6)

In [51]:
# let's get some values from this generator
# what message gets printed when?

print(next(my_gen))
print()


In generator function
- before yield
0



In [52]:
print(next(my_gen))
print()

- after yield
- before yield
1



---

**Ģeneratori var būt bezgalīgi** (piemēram, tie var ģenerēt bezgalīgas virknes):
- šādos gadījumos programmai, kas izmanto ģeneratoru, ir jāierobežo iterāciju skaits un jāpārliecinās, ka tās izpilde kaut kādā brīdī tiek pabeigta.

In [53]:
# Fibonacci sequence – an infinite generator

def fibonacci_gen():
    """
    Generates a Fibonacci sequence.
    """
    a, b = 0, 1

    while True:
        yield a
        a, b = b, a+b

In [54]:
# we can use islice() to limit the length of the iterator

# https://docs.python.org/3/library/itertools.html#itertools.islice
from itertools import islice

my_numbers = fibonacci_gen()
my_numbers = islice(my_numbers, 20)

for item in my_numbers:
    print(item)

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181


In [55]:
# you can use multiple yield statements
# ... but in this example you might as well use a list

def city_gen():
    yield "Rīga"
    # we could have done some processing here
    yield "Liepāja"
    yield "Valmiera"

for item in city_gen():
    print(item)

Rīga
Liepāja
Valmiera


---

#### 3.3. Ģeneratoru īspieraksts

Ģeneratoru īspieraksts (generator expressions) ir līdzīgs sarakstu īspierakstam (list comprehensions), izņemot to, ka tajā tiek izmantotas parastās iekavas `()`, nevis kvadrātiekavas `[]`.

Ģeneratoru īspieraksts izveido ģeneratora objektu, nevis sarakstu, un tas:
- izmanto "slinko" iterēšanu
- taupa atmiņu, neizveidojot visu sarakstu atmiņā

In [56]:
squares = [n**2 for n in range(50)]

print(sys.getsizeof(squares))
print()

print(squares)

472

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401]


In [57]:
squares_gen = (n**2 for n in range(50))

print(sys.getsizeof(squares_gen))
print(type(squares_gen))
print()

print(squares_gen)

200
<class 'generator'>

<generator object <genexpr> at 0x12c3cedc0>


In [58]:
print(list(squares_gen))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401]


In [59]:
print(sum(n**2 for n in range(50)))

40425


---

#### 3.4. Ģeneratori kā datu apstrādes plūsmas

[**Generator Tricks for Systems Programmers, v3.0**](https://www.dabeaz.com/generators/)
- autors: David M. Beazley
- apskata dažādus paņēmienus, kā pielietot ģeneratoru funkcijas un ģeneratoru īspierakstu sistēmas programmēšanas kontekstā (žurnālfailu apstrāde, teksta parsēšana u.t.t.)

https://github.com/dabeaz/generators

https://www.dabeaz.com/generators/Generators.pdf

Ģeneratoru funkcijas un ģeneratoru īspieraksts ir jaudīgi rīki lielu vai straumējamu (streaming) datu kopu apstrādei. To "slinkā" izpilde un atmiņas efektīvā izmantošana padara tos piemērotus tādiem uzdevumiem kā žurnālfailu apstrāde, teksta parsēšana un lielu datu plūsmu pārvaldība.

---

**Uzdevums**: Aprēķināt kopējo pārsūtīto baitu skaitu, summējot pēdējo datu kolonnu Apache tīmekļa servera žurnālā.

```
81.107.39.38 - ... "GET /favicon.ico HTTP/1.1" 404 133
81.107.39.38 - ... "GET /ply/bookplug.gif HTTP/1.1" 200 23903
81.107.39.38 - ... "GET /ply/ply.html HTTP/1.1" 200 97238
```

---

**Skat. David Beazley prezentāciju no 30. slaida**

```
with open("access-log") as wwwlog:

    bytecolumn = (line.rsplit(None,1)[1] for line in wwwlog)
    bytes_sent = (int(x) for x in bytecolumn if x != '-')
    print("Total", sum(bytes_sent))
```

**Šī datu apstrādes plūsma tika īstenota izmantojot ģeneratoru īspierakstu.**

```
open("access-log") => bytecolumn => bytes_sent => sum()
```

---

## Nodarbības pārskats

Šajā nodarbībā Jūs apguvāt šādas tēmas:

* Dekoratori - kas tie ir un kā tos pielietot
* Kontekstu pārvaldnieki - kas tie ir un kā tos pielietot
* Ģeneratori un ģeneratoru īspieraksts - kas tie ir un kā tos pielietot

## Praktiskie uzdevumi

### 1. uzdevums - Izveidojiet dekoratoru

Izveidojiet dekoratoru, kas nodrukā Jūsu vārdu pirms un pēc dekorētās funkcijas izsaukuma.

### 2. uzdevums - Izveidojiet konteksta pārvaldnieku

* Izveidojiet konteksta pārvaldnieku, kas nodrukā Jūsu vārdu pirms un pēc koda bloka izsaukuma.
* Papildiniet konteksta pārvaldnieku ar koda bloka izpildes laika uzskaiti.

### 3. uzdevums - Pielietojiet ģeneratorus datu apstrādē

Pielietojiet ģeneratorus lai izveidotu datu apstrādes plūsmu, kas sastāv no šādiem posmiem:

- atvērt teksta failu
- sadalīt nolasīto tekstu vārdos (izveidojot ģeneratoru, kas atgriež vārdus / tokenus)
- atfiltrēt vārdus, kas ir vairāk nekā 3 simbolus gari
- saskaitīt katra vārda pieminēšanas biežumu
- nodrukāt iegūto vārdu lietojuma statistiku

## Papildus resursi

### 1. tēma

- [Python Decorators](https://realpython.com/primer-on-python-decorators/) apmācības materiāls
- [@functools.wraps](https://docs.python.org/3/library/functools.html#functools.wraps) dekorators
- [Flask Quickstart](https://flask.palletsprojects.com/en/2.3.x/quickstart/)
- [Data Classes in Python 3.7+](https://realpython.com/python-data-classes/)

### 2. tēma

- [Context Manager](https://realpython.com/python-with-statement/)

### 3. tēma

- [Python Wiki: Generators](https://wiki.python.org/moin/Generators)
- [How to Use Generators and yield in Python](https://realpython.com/introduction-to-python-generators/) – Real Python